# Imports

In [1]:
import requests
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import re
from datetime import datetime
import pandas as pd
from connection_keys import connection, alchemy_connection

### Configurando a engine do banco

In [ ]:
engine = alchemy_connection('db_hardware')

### Setup para abrir o navegador com o Selenium e puxar a primeira página

In [2]:
#--| Setup
options = Options()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
options.add_argument('--disable-extensions')
#options.add_argument("--headless")
browser = webdriver.Chrome(executable_path=r'chromedriver.exe', options=options)

In [3]:

# --| Parse or automation
browser.get('https://www.kabum.com.br/hardware/placa-de-video-vga?pagina=1')
html = soup(browser.page_source, 'html.parser')
# browser.implicitly_wait(5)

### Configuiração para percorer as páginas e salvar todos dados com a taga referente ao produto em uma lista

In [4]:
pagina = 0
html_produtos = []
flag = 0

while flag == 0:
    items = html.findAll('div', {"class":"sc-fzqNqU jmuOAh"})
    if items != []:
        try:
            pagina = pagina + 1
            browser.get(f'https://www.kabum.com.br/hardware/placa-de-video-vga?pagina={pagina}')
            html = soup(browser.page_source, 'html.parser')
        except Exception: break
        html_produtos.append(items)
    else:
        break

### Tranformação da lista de listas em uma lista normal para percorer com mais facildiade os atributos do produto

In [5]:
flat_list = []
for sublist in html_produtos:
    for item in sublist:
        flat_list.append(item)

### Percorrendo a lista, capurando e salvando cada atributo no dicionario

In [6]:
now = datetime.now()
produtos = []

for container in flat_list:
    
    # Estreutura de para salvar
    marca = container.div.img['alt']
    nome_produto = container.find('a', {"class":"sc-fzoLsD gnrNhT item-nome"}).text

    if container.find('div', {"class":"sc-fzpjYC hKlXgM"}) != None:
        preco_antigo_cartao = (container.find('div', {"class":"sc-fzpjYC hKlXgM"}).text).split(" ")[2]
    else: 
        preco_antigo_cartao = '0'
    preco_atual_cartao =  (container.find('div', {"class":"sc-fznxsB ksiZrQ"}).text).split(" ")[1]
    preco_boleto = (container.find('div', {"class":"sc-fznWqX qatGF"}).text).split(" ")[1]
    
    image = container.find('div', {"class":"sc-fznKkj fEpBwY"})
    if "indisponivel" in image.img['src']:
        disponibilidade = 'Indisponivel'
    elif "disponivel" in image.img['src']:
        disponibilidade = 'Disponivel'

    produto = {
        "marca" : marca,
        "nome_produto": nome_produto,
        "preco_antigo_cartao": preco_antigo_cartao,
        "preco_atual_cartao": preco_atual_cartao,
        "preco_boleto": preco_boleto,
        "disponibilidade": disponibilidade,
        "data" : now.strftime("%Y-%m-%d"),
        "hora":  now.strftime("%H:%M:%S"),
        "loja": "Kabum" 
    }
    produtos.append(produto)


### Transformando o dict em Dataframe e visualizando uma amostra dos dados

In [7]:
df = pd.DataFrame(produtos)
df.sample(10)

,marca,nome_produto,preco_antigo_cartao,preco_atual_cartao,preco_boleto,disponibilidade,data,hora,loja
83,Asus,Placa de Vídeo Asus NVIDIA Dual GeForce RTX 20...,"5.882,24","5.411,65","4.599,90",Disponivel,2020-09-17,19:23:41,Kabum
174,Zotac,Placa de Vídeo Zotac Gaming NVIDIA GeForce GTX...,0,"1.529,29","1.299,90",Disponivel,2020-09-17,19:23:41,Kabum
12,Zotac,Placa de Vídeo Zotac NVIDIA GeForce GT 710 2GB...,"411,47","378,84","359,90",Disponivel,2020-09-17,19:23:41,Kabum
380,MSI,Placa de Vídeo MSI NVIDIA GeForce RTX 2070 Sup...,"5.411,65","4.235,18","3.599,90",Indisponivel,2020-09-17,19:23:41,Kabum
14,Evga,Placa de Vídeo EVGA NVIDIA GeForce RTX 2060 KO...,0,"2.823,41","2.399,90",Disponivel,2020-09-17,19:23:41,Kabum
93,Gigabyte,Placa de Vídeo Gigabyte NVIDIA GeForce GTX 165...,"1.949,89","1.882,24","1.599,90",Disponivel,2020-09-17,19:23:41,Kabum
66,Evga,Placa de Vídeo EVGA NVIDIA GeForce RTX 2080 Su...,"5.961,06","5.058,71","4.299,90",Disponivel,2020-09-17,19:23:41,Kabum
341,Galax,Placa de Vídeo Galax NVIDIA GeForce RTX 2080 T...,"11.114,00","9.411,65","7.999,90",Indisponivel,2020-09-17,19:23:41,Kabum
316,MSI,Placa de Vídeo MSI NVIDIA GeForce GTX 1050 Aer...,0,"752,82","639,90",Indisponivel,2020-09-17,19:23:41,Kabum
179,Galax,Placa de Vídeo Galax NVIDIA GeForce GTX 1660 S...,"2.599,88","2.352,82","1.999,90",Disponivel,2020-09-17,19:23:41,Kabum



### Criando a conexão com o banco e salvando os dados

In [8]:
df.to_sql('graphic_card', con=engine, if_exists='append', index = False)

----------------------------------------------------------------------------------------------------

In [181]:
#--| Parse or automation
browser.get('https://www.pichau.com.br/hardware/placa-de-video?p=1')
web = soup(browser.page_source, 'html.parser')

In [182]:
pagina = 0
html_produtos = []
flag = 0

while pagina != 9:
    items = web.findAll('div', {"class":"product-item-info"})
    if items != []:
        try:
            pagina = pagina + 1
            browser.get(f'https://www.pichau.com.br/hardware/placa-de-video?p={pagina}')
            web = soup(browser.page_source, 'html.parser')
        except Exception: break
        html_produtos.append(items)
    else:
        break

For the best experience on our site, be sure to turn on Javascript in your browser.
For the best experience on our site, be sure to turn on Javascript in your browser.
For the best experience on our site, be sure to turn on Javascript in your browser.
For the best experience on our site, be sure to turn on Javascript in your browser.
For the best experience on our site, be sure to turn on Javascript in your browser.
For the best experience on our site, be sure to turn on Javascript in your browser.
For the best experience on our site, be sure to turn on Javascript in your browser.
For the best experience on our site, be sure to turn on Javascript in your browser.
For the best experience on our site, be sure to turn on Javascript in your browser.


In [183]:
flat_list = []
for sublist in html_produtos:
    for item in sublist:
        flat_list.append(item)

In [241]:
def extract_brand(name):
    if 'AFOX' in item_aux.find('a', {"class":"product-item-link"}).text:
        return 'AFOX'
    elif 'AMD' in item_aux.find('a', {"class":"product-item-link"}).text:
        return 'AMD'
    elif 'AsRock' in item_aux.find('a', {"class":"product-item-link"}).text:
        return 'AsRock'
    elif 'Asus' in item_aux.find('a', {"class":"product-item-link"}).text:
        return 'Asus'
    elif 'EVGA' in item_aux.find('a', {"class":"product-item-link"}).text:
        return 'EVGA'
    elif 'Galax' in item_aux.find('a', {"class":"product-item-link"}).text:
        return 'Galax'
    elif 'Gigabyte' in item_aux.find('a', {"class":"product-item-link"}).text:
        return 'Gigabyte'
    elif 'MSI' in item_aux.find('a', {"class":"product-item-link"}).text:
        return 'MSI'
    elif 'PCYES!' in item_aux.find('a', {"class":"product-item-link"}).text:
        return 'PCYES'
    elif 'PNY' in item_aux.find('a', {"class":"product-item-link"}).text:
        return 'PNY'
    elif 'PowerColor' in item_aux.find('a', {"class":"product-item-link"}).text:
        return 'PowerColor'
    elif 'Sapphire' in item_aux.find('a', {"class":"product-item-link"}).text:
        return 'Sapphire'
    elif 'XFX' in item_aux.find('a', {"class":"product-item-link"}).text:
        return 'XFX'
    elif 'Sapphire' in item_aux.find('a', {"class":"product-item-link"}).text:
        return 'Sapphire'
    elif 'ZOGIS' in item_aux.find('a', {"class":"product-item-link"}).text:
        return 'ZOGIS'
    elif 'Zotac' in item_aux.find('a', {"class":"product-item-link"}).text:
        return 'Zotac'
    else:
        return ''

now = datetime.now()
produtos_pichau = []

for container in flat_list:
    if container.find('span', {"class":"price"}) != None:
        # Estreutura de para salvar
        marca = extract_brand(container.find('a', {"class":"product-item-link"}).text)
        nome_produto = container.find('a', {"class":"product-item-link"}).text

        preco_atual_cartao =  (container.find('span', {"class":"price"}).text)
        preco_boleto = (container.find('span', {"class":"price-boleto"}).text).split(" ")[2]
        
        disponibilidade = 'Disponivel'

        produto = {
            "marca" : marca,
            "nome_produto": nome_produto,
            "preco_antigo_cartao": None,
            "preco_atual_cartao": preco_atual_cartao,
            "preco_boleto": preco_boleto,
            "disponibilidade": disponibilidade,
            "data" : now.strftime("%Y-%m-%d"),
            "hora":  now.strftime("%H:%M:%S"),
            "loja": "Pichau" 
        }
        produtos_pichau.append(produto)

In [242]:
df_pichau = pd.DataFrame(produtos_pichau)
df_pichau['preco_atual_cartao'] = df_pichau['preco_atual_cartao'].replace({'R\$':''}, regex = True)
df_pichau['preco_boleto'] = df_pichau['preco_boleto'].replace({'R\$':''}, regex = True)
df_pichau['nome_produto'] = df_pichau['nome_produto'].replace({'\n':''}, regex = True)
df_pichau.sample(10)

,marca,nome_produto,preco_antigo_cartao,preco_atual_cartao,preco_boleto,disponibilidade,data,hora,loja
27,Galax,Placa de Video Asus Radeon RX 5600 XT 6GB GDDR...,None,"2.840,86","2.499,96",Disponivel,2020-09-19,17:49:11,Pichau
29,Galax,Placa de Video Galax GeForce GTX 1650 EX Plus ...,None,"1.249,97","1.099,97",Disponivel,2020-09-19,17:49:11,Pichau
34,Galax,Placa de VIdeo Gigabyte GeForce RTX 2070 Super...,None,"4.431,75","3.899,94",Disponivel,2020-09-19,17:49:11,Pichau
49,Galax,Placa de Video Gigabyte GeForce GTX 1650 D6 OC...,None,"1.373,85","1.208,99",Disponivel,2020-09-19,17:49:11,Pichau
22,Galax,Placa de Video ASRock Radeon RX 5700 XT 8GB Ph...,None,"3.465,86","3.049,96",Disponivel,2020-09-19,17:49:11,Pichau
18,Galax,Placa de Video PNY Quadro P4000 8GB GDDR5 256-...,None,"6.477,18","5.699,92",Disponivel,2020-09-19,17:49:11,Pichau
64,Galax,Placa de Video MSI Radeon RX 5700 XT 8GB Gamin...,None,"3.781,77","3.327,96",Disponivel,2020-09-19,17:49:11,Pichau
7,Galax,Placa de Video ASRock Radeon RX 5600 XT 6GB Ch...,None,"2.386,32","2.099,96",Disponivel,2020-09-19,17:49:11,Pichau
48,Galax,"Placa de Video Afox Radeon RX 550 2GB 128-bit,...",None,"636,25","559,90",Disponivel,2020-09-19,17:49:11,Pichau
75,Galax,Placa de Video Gigabyte GeForce RTX 2080 Super...,None,"4.899,99","4.311,99",Disponivel,2020-09-19,17:49:11,Pichau


In [244]:
df_pichau.to_sql('graphic_card', con=engine, if_exists='append', index = False)

_______________________________________________________

In [14]:
# containers = html.findAll('div', {"class":"sc-fzqNqU jmuOAh"})
# https://www.kabum.com.br/cgi-local/site/listagem/listagem.cgi?string=processador&btnG=&pagina=1&ordem=5&limite=30&prime=false&marcas=[%225%22,%226%22]&tipo_produto=[]&filtro=[]

In [15]:
# images = html.findAll('div', {"class":"sc-fznKkj fEpBwY"})

# # images.img['src']
# for image in images:
#     print(image.img['src'])
#     # if "indisponivel" in image.img['src']:
#     #     print('Indisponivel')
#     # elif "disponivel" in image.img['src']:
#     #     print('Disponivel')

In [16]:
# preco_antigo_cartao = float((container.find('div', {"class":"sc-fzpjYC hKlXgM"}).text).split(" ")[2].replace('.', '').replace(',', '.'))
# preco_atual_cartao =  float((container.find('div', {"class":"sc-fznxsB ksiZrQ"}).text).split(" ")[1].replace('.', '').replace(',', '.'))
# preco_boleto = float((container.find('div', {"class":"sc-fznWqX qatGF"}).text).split(" ")[1].replace('.', '').replace(',', '.'))


In [17]:
# print(preco_antigo_cartao)
# # print(re.search('^(([1-9]\d{0,2}(\.\d{3})*)|(([1-9]\.\d*)?\d))(\,\d\d)?', preco_atual_cartao).group())
# print(preco_atual_cartao)
# print(preco_boleto)

In [ ]:
# item_aux.find('span')
# nao_disponivel
# if "Indisponivel" not in item_aux.find('span').text:
#     print('Entrou aqui')
#     item_aux.find('a', {"class":"product-item-link"}).text
#     item_aux.find('span', {"class":"price"}).text
#     item_aux.find('span', {"class":"price-boleto"}).text.split(" ")[2]
# else:
#     print('Não ta')
# if item_aux.find('div', {"class":"price"}) != None:
#     print('Não ta')
# else:
#     print('Não ta2')